## 1. Import Python Libraries

In [ ]:
import os, sagemaker, subprocess, boto3, time

## 2. (OPTIONAL) Download Images from S3

In [ ]:
bucket_name = "uniben-data"
prefix = "yolo-validation/"
local_dir = "uniben-data/images/val"

if not os.path.isdir(local_dir):
    raise FileNotFoundError(f"Local directory not found: {local_dir}")
        
s3 = boto3.client('s3')
paginator = s3.get_paginator('list_objects_v2')

for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):
    for obj in page.get('Contents', []):
        key = obj['Key']
        if key.endswith('/'):
            continue  # Skip Folders
        filename   = os.path.basename(key)
        local_path = os.path.join(local_dir, filename)
        s3.download_file(bucket_name, key, local_path)
        print('.', end='')
print("\nDone downloading images!")

In [ ]:
import glob

def rename_files_remove_prefix(folder_path):
    """
    Rename files by removing everything before '__' in the filename
    
    Args:
        folder_path (str): Path to the folder containing files to rename
    """
    # Get all files in the folder
    files = glob.glob(os.path.join(folder_path, "*__*"))
    
    renamed_count = 0
    
    for file_path in files:
        # Get the directory and filename
        directory = os.path.dirname(file_path)
        old_filename = os.path.basename(file_path)
        
        # Check if filename contains '__'
        if '__' in old_filename:
            # Split by '__' and take everything after the first occurrence
            new_filename = '__'.join(old_filename.split('__')[1:])
            
            # Create new file path
            new_file_path = os.path.join(directory, new_filename)
            
            # Rename the file
            try:
                os.rename(file_path, new_file_path)
                print(f"Renamed: {old_filename} -> {new_filename}")
                renamed_count += 1
            except OSError as e:
                print(f"Error renaming {old_filename}: {e}")
    
    print(f"\nTotal files renamed: {renamed_count}")


folder_path = "uniben-data/labels/val"
rename_files_remove_prefix(folder_path)

## 3. TRAININGS

### 3.1. Install Ultralytics for YOLO11 model

https://docs.ultralytics.com/models/yolo11/#supported-tasks-and-modes

In [ ]:
!pip3 install albumentationsx ultralytics
from ultralytics import YOLO

model_name = 'yolo11x.pt'
model = YOLO(model_name)

### 3.2. Train the model

In [ ]:
train_start_time, a = time.strftime("%H:%M:%S", time.localtime()), time.localtime()
print("Start time:", train_start_time)

train_attempt="180_hung"
model.train(
            data="uniben-dataset.yaml",
            epochs=180,
            patience=0,
            imgsz=640,
            batch=10,
            # augment=True,
            # auto_augment="randaugment",
            exist_ok=True,
            project="uniben-trained",
            name=train_attempt,
            val=True,
            # cache=True,
            plots=True
           )

train_end_time, b = time.strftime("%H:%M:%S", time.localtime()), time.localtime()
print("End time:", train_end_time)

print(f"Training Time = {(time.mktime(b) - time.mktime(a))/60:0.2f} minutes.")

### Validation

In [ ]:
train_attempt="180_hung"
print(train_attempt)
model = YOLO(f'./uniben-trained/{train_attempt}/weights/best.pt')
metrics = model.val(
            data="uniben-dataset.yaml",
            project="uniben-val",
            name=train_attempt,
            save_json=True,
            iou=0.85,
            imgsz=640,
            batch=10,
            plots=True,
            save_txt=True,
            save_conf=True
)

In [ ]:
print("mAP50-95: ", metrics.box.map)  # mAP50-95
print("mAP50   : ", metrics.box.map50)  # mAP50
print("mAP75   : ", metrics.box.map75)  # mAP75
print("mean_results(): Mean of results, returns mp, mr, map50, map   : ", metrics.box.mean_results())  

print("AP at IoU thresholds from 0.5 to 0.95 for all classes  : ", metrics.box.ap) 
print("mp(): Mean precision of all classes   : ", metrics.box.mp)
print("mr(): Mean recall of all classes   : ", metrics.box.mr)  

print("list of mAP50-95 for each category: ", metrics.box.maps)  # list of mAP50-95 for each category

In [ ]:
print(metrics.summary()) 

### 3.3. (OPTIONAL) Resume training from the lastest epoch

### Get the latest train folder

In [ ]:
runs_dir = "runs/detect"
train_dirs = [
    d for d in os.listdir(runs_dir)
    if os.path.isdir(os.path.join(runs_dir, d)) and d.startswith("train")
]

latest_train_attempt = max(
    train_dirs,
    key=lambda d: os.path.getmtime(os.path.join(runs_dir, d))
)

print("Latest train folder: ", latest_train_attempt)

### Resume the training process of the latest train

In [ ]:
model = YOLO(f'runs/detect/{latest_train_attempt}/weights/last.pt')
model.train(resume=True)

### 3.4. (IMPORTANT, OPTIONAL) Local Inference

### Get the latest best.pt from latest train, put it into latest-best-model

In [ ]:
import shutil

runs_dir   = "runs/detect"            
target_dir = "latest-best-model"   

candidates = []
for name in os.listdir(runs_dir):
    path = os.path.join(runs_dir, name)
    if os.path.isdir(path) and name.startswith("train"):
        candidates.append(path)

if not candidates:
    raise RuntimeError(f"No 'train*' subfolders in {runs_dir}")

latest = max(candidates, key=lambda p: os.path.getmtime(p))
print(f"Latest train folder: {latest}")

src = os.path.join(latest, "weights", "best.pt")
dst = os.path.join(target_dir, "best.pt")
shutil.copy2(src, dst)
print(f"Copied:\n  {src}\n→ {dst}")

### Create local model to invoke

In [ ]:
!pip install opencv-python
import cv2, numpy as np, matplotlib.pyplot as plt, random
import base64, json

In [ ]:
train_attempt="180_hung"
print(train_attempt)
model = YOLO(f'./uniben-trained/{train_attempt}/weights/best.pt')

In [ ]:
img_path = 'images-test/z6686604803522_8d40b41b8489d82e13f15d9db092d1d8.jpg'
result = model.predict(
    source=img_path,
    save=False,
)

print(result)

In [ ]:
orig_image = cv2.imread(img_path)

CLASS_COLORS = {
    0: (0,255,0),
    1: (255,0,0),
    2: (0,0,255),
    3: (0,0,0),
}

CLASS_NAMES = {
    0: "abben",
    1: "boncha",
    2: "joco",
    3: "shelf",
}

for r in result:  # results is a list
    boxes = r.boxes.xyxy.cpu().numpy()
    classes = r.boxes.cls.cpu().numpy().astype(int)
    for (x1, y1, x2, y2), classID in zip(boxes, classes):
        color = CLASS_COLORS.get(classID, (255, 255, 255))
        cv2.rectangle(
            orig_image,
            (int(x1), int(y1)),
            (int(x2), int(y2)),
            color,
            thickness=2
        )

plt.figure(figsize=(12,8))
plt.imshow(cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

out_path = 'output_with_boxes.jpg'
cv2.imwrite(out_path, orig_image)
print(f"Annotated image saved to {out_path}")


### Run prediction for all validation images

In [ ]:
folder_path = 'uniben-data/images/val/'
all_entries = os.listdir(folder_path)

all_images_test = [f for f in all_entries
              if os.path.isfile(os.path.join(folder_path, f))]

print(all_images_test)

In [ ]:
for img in all_images_test:
    img_path = f'uniben-data/images/val/{img}'
    result = model.predict(
        source=img_path,
        save=False,
    )

    orig_image = cv2.imread(img_path)

    CLASS_COLORS = {
        0: (0,255,0),
        1: (255,0,0),
        2: (0,0,255),
        3: (0,0,0),
    }

    for r in result:  # results is a list
        boxes = r.boxes.xyxy.cpu().numpy()
        classes = r.boxes.cls.cpu().numpy().astype(int)
        scores  = r.boxes.conf.cpu().numpy()
        for (x1, y1, x2, y2), classID, score in zip(boxes, classes, scores):
            color = CLASS_COLORS.get(classID, (255, 255, 255))
            cv2.rectangle(
                orig_image,
                (int(x1), int(y1)),
                (int(x2), int(y2)),
                color,
                thickness=2
            )
            # Conf score
            label = f"{score:.2f}"
            (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            cv2.rectangle(
                orig_image,
                (int(x1), int(y1) - h - 4),
                (int(x1) + w, int(y1)),
                color,
                cv2.FILLED
            )
            cv2.putText(
                orig_image,
                label,
                (int(x1), int(y1) - 2),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255,255,255),
                thickness=1,
                lineType=cv2.LINE_AA
            )

    # plt.figure(figsize=(12,8))
    # plt.imshow(cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # plt.show()

    out_path = f'images-test-annotated/{img}'
    cv2.imwrite(out_path, orig_image)
print("Done")

# -----------------------------
# Deploy endpoint

## 4. Zip the code and model into `model.tar.gz` and upload it to specific S3 bucket
Here permission is granted to the S3 bucket created with CDK and not any other bucket

Vào trong folder train của mô hình muốn deploy, copy best.pt ra ngoài

In [ ]:
# Copy file from source folder to destination folder
import shutil
train_attempt="180_hung"
source_file = f"uniben-trained/{train_attempt}/weights/best.pt"
destination_folder = "./"

# Copy the file (preserves metadata)
shutil.copy2(source_file, destination_folder)

In [ ]:
model_zip = 'best.pt'
code_zip = "code/"
bashCommand = f"tar -cpzf  model.tar.gz {model_zip} {code_zip}"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

## 5. Select yolo11 bucket

In [ ]:
s3_client = boto3.client('s3')
response = s3_client.list_buckets()
for bucket in response['Buckets']:
    if 'yolo11' in bucket["Name"]:
        bucket = 's3://' + bucket["Name"]
        break
print(f'Bucket: {bucket}')

## 6. Upload model to S3

In [ ]:
from sagemaker import s3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
print(f'Account ID: {account_id}')
role = get_execution_role()
print(f'Role: {role}')

prefix = "yolo11"
model_data = s3.S3Uploader.upload("model.tar.gz", bucket + "/" + prefix)
print(f'Model Data: {model_data}')

## 7. Create the SageMaker PyTorchModel

In [ ]:
from sagemaker.pytorch import PyTorchModel

sess = sagemaker.Session(default_bucket=bucket.split('s3://')[-1])
model = PyTorchModel(entry_point='inference.py',
                     model_data=model_data, 
                     framework_version='1.12', 
                     py_version='py38',
                     role=role,
                     env={'TS_MAX_RESPONSE_SIZE':'20000000', 'YOLO11_MODEL': 'best.pt'},
                     sagemaker_session=sess)

## 8. Deploy the model on SageMaker Endpoint:

In [ ]:
print(time.strftime("%Y-%m-%d-%H-%M", time.localtime()))

In [ ]:
from sagemaker.deserializers import JSONDeserializer

INSTANCE_TYPE = 'ml.c5.xlarge'
ENDPOINT_NAME = 'yolo11-pytorch-' + str(time.strftime("%Y-%m-%d-%H-%M", time.localtime()))
print(ENDPOINT_NAME)
model.deploy(initial_instance_count=1, 
             instance_type=INSTANCE_TYPE,
             deserializer=JSONDeserializer(),
             endpoint_name=ENDPOINT_NAME)

## 9.(OPTIONAL) Cleanup by removing Endpoint, Endpoint Config and Model

In [ ]:
response = sm_client.describe_endpoint_config(EndpointConfigName=ENDPOINT_NAME)
print(response)
endpoint_config_name = response['EndpointConfigName']

# Delete Endpoint
sm_client.delete_endpoint(EndpointName=ENDPOINT_NAME)

# Delete Endpoint Configuration
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Delete Model
for prod_var in response['ProductionVariants']:
    model_name = prod_var['ModelName']
    sm_client.delete_model(ModelName=model_name)     

In [ ]:
import boto3
import os
from pathlib import Path

def upload_folder_to_s3(local_folder, bucket_name, s3_folder_prefix=""):
    """
    Upload a local folder to S3 bucket
    
    Args:
        local_folder (str): Path to local folder to upload
        bucket_name (str): Name of S3 bucket
        s3_folder_prefix (str): Prefix for S3 keys (folder path in bucket)
    """
    
    s3_client = boto3.client('s3')
    print(1)
    # Walk through all files in the local folder
    for root, dirs, files in os.walk(local_folder):
        for file in files:
            local_path = os.path.join(root, file)
            
            # Calculate relative path from the base folder
            relative_path = os.path.relpath(local_path, local_folder)
            
            # Create S3 key (object name)
            if s3_folder_prefix:
                s3_key = f"{s3_folder_prefix}/{relative_path}".replace("\\", "/")
            else:
                s3_key = relative_path.replace("\\", "/")
            
            try:
                print(f"Uploading {local_path} to s3://{bucket_name}/{s3_key}")
                s3_client.upload_file(local_path, bucket_name, s3_key)
            except Exception as e:
                print(f"Error uploading {local_path}: {e}")

# Example usage
upload_folder_to_s3(
    local_folder="./uniben-data/labels/val",
    bucket_name="uniben-data",
    s3_folder_prefix="hung_lambda/labels/val" 
)

# TEST

In [ ]:
# build_and_push.py
import boto3
import subprocess
import sys
import os
from pathlib import Path

def get_account_id():
    """Get AWS account ID"""
    sts = boto3.client('sts')
    return sts.get_caller_identity()['Account']

def get_region():
    """Get AWS region"""
    session = boto3.Session()
    return session.region_name

def create_ecr_repository(repository_name, region):
    """Create ECR repository if it doesn't exist"""
    ecr = boto3.client('ecr', region_name=region)
    
    try:
        ecr.describe_repositories(repositoryNames=[repository_name])
        print(f"Repository {repository_name} already exists")
    except ecr.exceptions.RepositoryNotFoundException:
        ecr.create_repository(repositoryName=repository_name)
        print(f"Created repository {repository_name}")

def build_and_push_docker_image():
    """Build and push Docker image to ECR"""
    
    # Configuration
    account_id = get_account_id()
    region = get_region()
    repository_name = "yolo11-training"
    image_tag = "latest"
    
    # Full image URI
    image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{repository_name}:{image_tag}"
    
    print(f"Account ID: {account_id}")
    print(f"Region: {region}")
    print(f"Repository: {repository_name}")
    print(f"Image URI: {image_uri}")
    
    # Create ECR repository
    create_ecr_repository(repository_name, region)
    
    # Get ECR login token
    print("Getting ECR login token...")
    ecr = boto3.client('ecr', region_name=region)
    auth_token = ecr.get_authorization_token()['authorizationData'][0]['authorizationToken']
    
    # Login to ECR
    print("Logging in to ECR...")
    login_cmd = f"aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com"
    subprocess.run(login_cmd, shell=True, check=True)
    
    # Build Docker image
    print("Building Docker image...")
    build_cmd = f"docker build -t {repository_name} ."
    subprocess.run(build_cmd, shell=True, check=True)
    
    # Tag the image
    print("Tagging image...")
    tag_cmd = f"docker tag {repository_name}:latest {image_uri}"
    subprocess.run(tag_cmd, shell=True, check=True)
    
    # Push to ECR
    print("Pushing image to ECR...")
    push_cmd = f"docker push {image_uri}"
    subprocess.run(push_cmd, shell=True, check=True)
    
    print(f"\nDocker image pushed successfully!")
    print(f"Image URI: {image_uri}")
    
    return image_uri

if __name__ == "__main__":
    # Ensure we're in the right directory
    if not os.path.exists("Dockerfile"):
        print("Error: Dockerfile not found in current directory")
        print("Please run this script from the directory containing the Dockerfile")
        sys.exit(1)
    
    image_uri = build_and_push_docker_image()

In [ ]:
docker build -t yolo .

In [ ]:
cp -r ./uniben-data/images/train/* local_test/input/data/train/images/
cp -r ./uniben-data/labels/train/* local_test/input/data/train/labels/
cp -r ./uniben-data/images/val/* local_test/input/data/validation/images/
cp -r ./uniben-data/labels/val/* local_test/input/data/validation/labels/

In [ ]:
docker run --rm -it \
  --gpus all \
  -v $(pwd)/local_test/input/data:/opt/ml/input/data \
  -v $(pwd)/local_test/model:/opt/ml/model \
  -v $(pwd)/local_test/output:/opt/ml/output \
  -e SM_MODEL_DIR=/opt/ml/model \
  -e SM_CHANNEL_TRAIN=/opt/ml/input/data/train \
  -e SM_CHANNEL_VALIDATION=/opt/ml/input/data/validation \
  -e SM_OUTPUT_DATA_DIR=/opt/ml/output/data \
  yolo \
  /bin/bash